In [1]:
import os
import sys
import json
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from typing import List, Tuple, Dict, Any, Optional

In [2]:
PATH_ROOT_DIR: str = ".."

INFO_TO_USE_FOR_CLUSTERING: List[str] = [
    'exception_message', 'essential_file_content']

PCA_COMPONENTS: int = 3

AI_MODEL: str = "llama-3.1-70b-versatile"  # or "gpt-4o-mini"

N_POSITIVE_EXAMPLES_TO_SHOW_LLM: int = 5

# Test run
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__14_48__qiskit")
# Run with 10k programs: v001.yaml
# program_bank/v001/2024_10_04__15_10__qiskit
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__15_10__qiskit")
# program_bank/v002/2024_10_07__17_03__qiskit
# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__qiskit")

# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__sample_errors")


# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__qiskit")
# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__sample_errors")

# program_bank/v005/2024_10_22__22_32__qiskit
DIR_PROGRAM_BANK: str = os.path.join(
    "..", "program_bank", "v005", "2024_10_22__22_32__qiskit")
DIR_SAMPLE_ERRORS: str = os.path.join(
    "..", "program_bank", "v005",
    "2024_10_22__22_32__sample_errors_no_pennylane")


PROMPT_CONTENT: str = "BOTH_ERROR_AND_PROGRAM"

In [3]:
# print the current folder
print(f"Current folder: {os.getcwd()}")

all_global_vars = dict(globals().copy())
# print all global variables
for k, v in all_global_vars.items():
    if k.isupper():
        print(f"{k}: {v}")

Current folder: /home/paltenmo/projects/crossplatform/notebooks
PATH_ROOT_DIR: ..
INFO_TO_USE_FOR_CLUSTERING: ['exception_message', 'essential_file_content']
PCA_COMPONENTS: 3
AI_MODEL: llama-3.1-70b-versatile
N_POSITIVE_EXAMPLES_TO_SHOW_LLM: 5
DIR_PROGRAM_BANK: ../program_bank/v005/2024_10_22__22_32__qiskit
DIR_SAMPLE_ERRORS: ../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane
PROMPT_CONTENT: BOTH_ERROR_AND_PROGRAM


In [4]:
import openai
import groq
PATH_GROQ_TOKEN = os.path.join(PATH_ROOT_DIR, "groq_token.txt")
GROQ_TOKEN = Path(PATH_GROQ_TOKEN).read_text().strip()

# Export the GROQ_TOKEN as GROQ_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_TOKEN

PATH_OPENAI_API_KEY = os.path.join(PATH_ROOT_DIR, "openai_token.txt")
OPENAI_API_KEY = Path(PATH_OPENAI_API_KEY).read_text().strip()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
# count generated programs in the program bank
count_generated_programs = len(list(Path(DIR_PROGRAM_BANK).rglob("*.py")))
print(f"Number of generated programs: {count_generated_programs}")

# count the number of errors
count_errors = len(list(Path(DIR_PROGRAM_BANK).rglob("*_error.json")))
print(f"Number of errors: {count_errors}")

Number of generated programs: 82166
Number of errors: 189431


## Read Files

In [6]:
from multiprocessing import Pool
import re


def read_json_file(file_path: str) -> Dict[str, Any]:
    with open(file_path, 'r') as file:
        data = json.load(file)
        data["error_file"] = os.path.basename(file_path)
        return data


def read_all_programs(dir_path: str) -> pd.DataFrame:
    json_files = [os.path.join(dir_path, f) for f in os.listdir(
        dir_path) if re.match(r'.*_error\.json$', f)]
    records = []

    with Pool() as pool:
        results = pool.map(read_json_file, json_files)
        records.extend(results)

    return pd.DataFrame(records)


# Use the function to read all programs
df_errors = read_all_programs(DIR_PROGRAM_BANK)

# augment the dataframe with the proram content


def read_file_content(file_name: str) -> str:
    file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
    with open(file_path, 'r') as file:
        return file.read()


# Use parallel apply to read the file content
with Pool() as pool:
    df_errors['file_content'] = pool.map(
        read_file_content, df_errors['current_file'])

# Alternatively, you can use apply without parallel processing
# df_errors['file_content'] = df_errors['current_file'].apply(read_file_content)

df_errors.head()
n_errors = len(df_errors)
print(f"Number of errors: {n_errors}")

Number of errors: 189431


# Exclude Pennylane (errors)

In [7]:
# keep only those without pennylane in the involeved_functions
# cast the involved_functions to string
df_errors['involved_functions'] = df_errors['involved_functions'].astype(
    str)
df_errors = df_errors[
    ~df_errors['involved_functions'].str.contains('pennylane')]
n_errors = len(df_errors)
print(f"Number of errors without PennyLane: {n_errors}")

Number of errors without PennyLane: 54235


# Minimize Sample

In [8]:
import shutil
SAMPLE_SIZE = 10

# keep only errors with `The circuits are not equivalent` in the exception message
df_errors = df_errors[df_errors['exception_message'].str.contains(
    'The circuits are not equivalent')]

# pick a sample of error files
sample_errors = df_errors.sample(SAMPLE_SIZE, random_state=42)

# get the paths of the error files
sample_errors_paths = [os.path.join(
    DIR_PROGRAM_BANK, f) for f in sample_errors['error_file']]


# copy them to a new folder
os.makedirs(DIR_SAMPLE_ERRORS, exist_ok=True)
for file_path in sample_errors_paths:
    shutil.copy(file_path, DIR_SAMPLE_ERRORS)
    print(f"Copying {file_path} to {DIR_SAMPLE_ERRORS}")

Copying ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_77145_ac209a_f3f76a_error.json to ../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane
Copying ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_72419_2484a1_565431_error.json to ../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane
Copying ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_74712_c84e32_db5fb1_error.json to ../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane
Copying ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_77413_d0ed80_3f8617_error.json to ../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane
Copying ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_49850_79a6d5_cbe4b9_error.json to ../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane
Copying ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_70298_e19d16_a2791b_error.j

In [9]:
import sys
sys.path.append("..")

In [10]:
from analysis_and_reporting.ddmin_target_file import minimize_instructions
from pathlib import Path

error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, f)
    for f in os.listdir(DIR_SAMPLE_ERRORS)
    if re.match(r'.*_error\.json$', f)
]

# run minimization command:
# python -m analysis_and_reporting.ddmin_target_file --input_folder {folder_with_files} --path_to_error {path_to_json_file} --clue_message 'The circuits are not equivalent'

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)

# add the parent folder to the path
sys.path.append("..")

for error_path in error_paths_in_sample_folder:
    path_minimized_file = error_path.replace("_error.json", "_error_min.py")
    # check if the same file exists in the sample folder with extension _min.py
    # if yes then skip
    if os.path.exists(path_minimized_file):
        print(f"Skipping minimization for {error_path}")
        continue

    print(f"Running minimization for {error_path}")
    minimize_instructions(
        input_folder=Path(DIR_PROGRAM_BANK),
        path_to_error=Path(error_path),
        clue_message='The circuits are not equivalent'
    )

    # copy the minimized file from DIR_PROGRAM_BANK to the sample folder
    name_minimized_file = os.path.basename(path_minimized_file)
    shutil.copy(
        os.path.join(DIR_PROGRAM_BANK, name_minimized_file),
        path_minimized_file
    )

Error paths in sample folder (10):
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_77145_ac209a_f3f76a_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_77413_d0ed80_3f8617_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_70298_e19d16_a2791b_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_18690_1e7c44_8e94c0_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_72419_2484a1_565431_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_74712_c84e32_db5fb1_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_15090_228b8a_a65e6b_error.json
../program_bank/v005/2024_10_22__22_32__sample_errors_no_pennylane/qiskit_circuit_32q_10g_49850_79a6d5_cbe4b9_error.js

[16:48:40] Involved functions:                                                             ]8;id=593167;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=930091;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=710492;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=990985;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=924703;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=897092;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_qiskit                                                  ]8;id=681954;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=314491;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

...

             - compare_qasm_via_qcec                                                       ]8;id=841619;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=839481;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=617634;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=185827;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_qiskit                                                  ]8;id=859633;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=575699;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

Output: Saved the Pytket circuit to /workspace/to_execute_qc_pytket.qasm
Saved the Qiskit circuit to /workspace/to_execute_qc_qiskit.qasm
Exported QASM files: [('/workspace/to_execute_qc_pytket.qasm', 'export_to_qasm_with_pytket'), ('/workspace/to_execute_qc_qiskit.qasm', 'export_to_qasm_with_qiskit')]
The circuits are equivalent: /workspace/to_execute_qc_pytket.qasm, /workspace/to_execute_qc_qiskit.qasm

Program to_execute.py executed successfully.
Output: Saved the Pytket circuit to /workspace/to_execute_qc_pytket.qasm
Saved the Qiskit circuit to /workspace/to_execute_qc_qiskit.qasm
Exported QASM files: [('/workspace/to_execute_qc_pytket.qasm', 'export_to_qasm_with_pytket'), ('/workspace/to_execute_qc_qiskit.qasm', 'export_to_qasm_with_qiskit')]
Log has been saved to /workspace/to_execute_b3d19a_error.json

Program to_execute.py executed successfully.
New exception message: The circuits are not equivalent: /workspace/to_execute_qc_pytket.qasm, /workspace/to_execute_qc_qiskit.qasm
Out

[16:55:46] Minimized file saved as:                                                        ]8;id=121983;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=190083;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#240\240]8;;\
           ../program_bank/v005/2024_10_22__22_32__qiskit/qiskit_circuit_32q_10g_3311_1210                         
           5c_ac774b_error_min.py                                                                                  

In [12]:
# # copy all files ending with _min.py in the program bank to the sample folder
# min_files = [f for f in os.listdir(DIR_PROGRAM_BANK) if f.endswith("_min.py")]
# for file_path in min_files:
#     shutil.copy(os.path.join(DIR_PROGRAM_BANK, file_path), DIR_SAMPLE_ERRORS)
#     print(f"Copying {file_path} to {DIR_SAMPLE_ERRORS}")

In [15]:
# read all the error files in the sample folder
df_errors_sample = read_all_programs(DIR_SAMPLE_ERRORS)

# print their content between `# <START_GATES>`` and `qc.measure``


def extract_essential_content(file_path: str) -> str:
    with open(file_path, 'r') as file:
        content = file.read()
        start_tag = "# <START_GATES>"
        start = content.find(start_tag) + len(start_tag)
        end = content.find("qc.measure")
        return content[start:end]


df_errors_sample["essential_file_content"] = df_errors_sample["error_file"].apply(
    lambda x: extract_essential_content(
        os.path.join(DIR_SAMPLE_ERRORS, x.replace("_error.json", "_error_min.py"))))
# keep only those without pennylane in the involeved_functions
# cast the involved_functions to string
df_errors_sample['involved_functions'] = df_errors_sample['involved_functions'].astype(
    str)
df_errors_sample = df_errors_sample[
    ~df_errors_sample['involved_functions'].str.contains('pennylane')]

for i, row in df_errors_sample.iterrows():
    print('='*50)
    print(
        f"Minimized filepath: {os.path.join(DIR_SAMPLE_ERRORS, row['error_file'].replace('_error.json', '_error_min.py'))}")
    print(
        f"Error {i}:\n{row['exception_message']}\n{row['essential_file_content']}\n")
    print(f"Involved functions: {row['involved_functions']}\n")

df_errors_sample.head()

FileNotFoundError: [Errno 2] No such file or directory: '../program_bank/v005/2024_10_22__22_32__sample_errors/qiskit_circuit_32q_10g_49802_899c61_524cf8_error_min.py'